Business Data Scientist Intern Online Assessment - Maxwell Thomson

### Import Packages and CSV Files

In [1]:
import datetime
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:.2f}'.format
import warnings
warnings.filterwarnings("ignore")

In [2]:
booking = pd.read_csv('booking.csv')
booking_status = pd.read_csv('booking_status.csv')
currency = pd.read_csv('currency.csv')
exchange = pd.read_csv('exchange.csv')

### Quick Overview

In [3]:
booking2 = booking.copy()
booking.head()

,booking_received_timestamp,booking_status,booking_check_in,booking_check_out,booking_earned,booking_currency
0,2019-10-07 14:31:03,1,2019-11-01,2019-11-03,1164000.00,1
1,2019-09-12 21:36:36,1,2019-11-01,2019-11-02,783515.00,1
2,2019-09-17 10:00:37,1,2019-11-01,2019-11-04,1484999.00,1
3,2019-10-16 09:46:50,1,2019-11-01,2019-11-05,1741072.00,1
4,2019-09-13 17:47:08,1,2019-11-01,2019-11-02,48.00,2


In [4]:
booking_status

,booking_status,booking_status_desc
0,1,Confirmed
1,2,Cancelled


In [5]:
currency

,currency_code,currency_label,currency_desc
0,1,Rp,IDR
1,2,$,USD
2,3,?,EUR
3,4,AUD,AUD


In [6]:
exchange2 = exchange.copy()
exchange.tail()

,id,date,base_currency,sgd,eur,aud,idr
147,2020-03-27,2020-03-27 22:29:24,USD,1.43,0.91,1.65,16337.50
148,2020-03-28,2020-03-28 22:29:24,USD,1.43,0.90,1.62,16019.96
149,2020-03-29,2020-03-29 22:29:24,USD,1.43,0.90,1.62,16019.96
150,2020-03-30,2020-03-30 22:29:26,USD,1.42,0.91,1.63,16030.40
151,2020-03-31,2020-03-31 22:29:29,USD,1.43,0.91,1.64,16030.40


# Kuy

### Convert to datetime

In [7]:
booking2[['booking_received_timestamp','booking_check_in','booking_check_out']] = booking[['booking_received_timestamp','booking_check_in','booking_check_out']].apply(pd.to_datetime, errors='coerce')
exchange2[['id','date']] = exchange[['id','date']].apply(pd.to_datetime, errors='coerce')

### Drop invalid datetime, filter confirmed booking_status, create new columns

In [8]:
#Check invalid datetime
print(booking['booking_received_timestamp'].min()) 
print(booking2.isna().sum()) #booking's invalid datetime has been converted to NaN in booking2

0000-00-00 00:00:00
booking_received_timestamp    12
booking_status                 0
booking_check_in               0
booking_check_out              0
booking_earned                 0
booking_currency               0
dtype: int64


In [9]:
booking2 = booking2.dropna() #drop invalid datetime
booking2 = booking2[booking2.booking_status == 1] #bookings that are cancelled generates no revenue
booking2 = booking2.sort_values(by='booking_check_in').reset_index(drop=True) #sort dates by check_in dates
booking2['day_of_stay'] = booking2['booking_check_out'] - booking2['booking_check_in'] #total number of days stayed
booking2['avg_revenue'] = booking2['booking_earned']/booking2['day_of_stay'].dt.days #average booking revenue

### Currency converted to USD (Submitted as CSV)

In [10]:
#Create new dataframe
interval = booking2['booking_check_out'].max() - booking2['booking_check_in'].min()
revenue_per_day = pd.DataFrame(np.zeros((interval.days, 2)), index=np.arange(interval.days), columns = ['date', 'revenue'])

#Generate dates for new dataframe's dates
rng = pd.date_range(booking2['booking_check_in'].min(), booking2['booking_check_out'].max()-datetime.timedelta(days=1))
rng = rng.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
unique_date = pd.DataFrame(rng, columns=['date'])
unique_date['date'] = unique_date['date'].apply(pd.to_datetime, errors='coerce')
revenue_per_day['date'] = unique_date['date']

In [11]:
booking2['booking_currency'].value_counts()

1    8370
2     948
Name: booking_currency, dtype: int64

In [12]:
for i in range(booking2.shape[0]):
  day_start = booking2['booking_check_in'].iloc[i]
  days_num = booking2['day_of_stay'].iloc[i].days
  avg_revenue = booking2['avg_revenue'].iloc[i]   #Initialize day_start, days_num, avg revenue for every row(booking)
  for j in range(days_num):  #Iterate through each stayed day for each booking
    day_add = day_start + datetime.timedelta(days=j)  #update day
    update = revenue_per_day[revenue_per_day['date'] == day_add]
    if booking2['booking_currency'].iloc[i]==1:
      conv_avg = avg_revenue/exchange2[exchange2['id'] == day_add]['idr']  #Convert to USD based on 'exchange' currency rate
      revenue_per_day.loc[update.index, 'revenue' ] += conv_avg 
    else:
      revenue_per_day.loc[update.index, 'revenue' ] += avg_revenue   #update revenue

In [13]:
revenue_per_day

,date,revenue
0,2019-11-01,4219.44
1,2019-11-02,7647.67
2,2019-11-03,7399.31
3,2019-11-04,7410.97
4,2019-11-05,8159.75
...,...,...
146,2020-03-26,970.65
147,2020-03-27,851.37
148,2020-03-28,796.60
149,2020-03-29,535.03


### Currency converted to IDR

In [14]:
#Create new dataframe
interval = booking2['booking_check_out'].max() - booking2['booking_check_in'].min()
revenue_per_day2 = pd.DataFrame(np.zeros((interval.days, 2)), index=np.arange(interval.days), columns = ['date', 'revenue'])

rng = pd.date_range(booking2['booking_check_in'].min(), booking2['booking_check_out'].max()-datetime.timedelta(days=1))
rng = rng.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
unique_date = pd.DataFrame(rng, columns=['date'])
unique_date['date'] = unique_date['date'].apply(pd.to_datetime, errors='coerce')
revenue_per_day2['date'] = unique_date['date']

In [15]:
for i in range(booking2.shape[0]):
  day_start = booking2['booking_check_in'].iloc[i]
  days_num = booking2['day_of_stay'].iloc[i].days
  avg_revenue = booking2['avg_revenue'].iloc[i]
  for j in range(days_num):
    day_add = day_start + datetime.timedelta(days=j)
    update = revenue_per_day2[revenue_per_day2['date'] == day_add]

    if booking2['booking_currency'].iloc[i]==2:
      conv_avg = avg_revenue*exchange2[exchange2['id'] == day_add]['idr']
      revenue_per_day2.loc[update.index, 'revenue' ] += conv_avg
    else:
      revenue_per_day2.loc[update.index, 'revenue' ] += avg_revenue

In [16]:
revenue_per_day2

,date,revenue
0,2019-11-01,59203396.59
1,2019-11-02,107007763.79
2,2019-11-03,103511216.28
3,2019-11-04,103912977.26
4,2019-11-05,114850614.13
...,...,...
146,2020-03-26,15593533.28
147,2020-03-27,13909318.16
148,2020-03-28,12761574.67
149,2020-03-29,8571162.67


## Currency not converted

In [17]:
#Create new dataframe
interval = booking2['booking_check_out'].max() - booking2['booking_check_in'].min()
revenue_per_day3 = pd.DataFrame(np.zeros((interval.days, 2)), index=np.arange(interval.days), columns = ['date', 'revenue'])

rng = pd.date_range(booking2['booking_check_in'].min(), booking2['booking_check_out'].max()-datetime.timedelta(days=1))
rng = rng.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
unique_date = pd.DataFrame(rng, columns=['date'])
unique_date['date'] = unique_date['date'].apply(pd.to_datetime, errors='coerce')
revenue_per_day3['date'] = unique_date['date']

In [18]:
for i in range(booking2.shape[0]):
  day_start = booking2['booking_check_in'].iloc[i]
  days_num = booking2['day_of_stay'].iloc[i].days
  avg_revenue = booking2['avg_revenue'].iloc[i]
  for j in range(days_num):
    day_add = day_start + datetime.timedelta(days=j)
    update = revenue_per_day3[revenue_per_day3['date'] == day_add]
    revenue_per_day3.loc[update.index, 'revenue' ] += avg_revenue

In [19]:
revenue_per_day3 #IDR mix with USD

,date,revenue
0,2019-11-01,53350511.55
1,2019-11-02,89593883.21
2,2019-11-03,83827846.38
3,2019-11-04,82588931.75
4,2019-11-05,92607386.34
...,...,...
146,2020-03-26,14690507.00
147,2020-03-27,12657452.16
148,2020-03-28,11534041.46
149,2020-03-29,7343629.46


#### Validation

In [20]:
booking_val = booking2.copy()
random_date = '2020-02-20' #Change date here

try:
  random_date_query = random_date.replace('-', '')
  selected_booking_val = booking_val.query('@random_date_query < booking_check_out & @random_date_query >= booking_check_in')
  grouped_booking = revenue_per_day3[revenue_per_day3['date']==random_date]['revenue']

  ori_sum = round(selected_booking_val['avg_revenue'].sum(), 2)
  grouped_sum = round(grouped_booking.values[0], 2)
  print(ori_sum==grouped_sum)

except (IndexError, ValueError):
  print("Select date from 2019-11-01 - 2020-03-30 !")

True


# Notes

There are 216 bookings with booking_status = 2 (cancelled booking) but having a non-zero revenue. It is assumed in this notebook that all booking with cancelled status gains no revenue at all. 

In [21]:
booking.loc[(booking['booking_status']==2) & (booking['booking_earned']!=0)]

,booking_received_timestamp,booking_status,booking_check_in,booking_check_out,booking_earned,booking_currency
11,2019-11-01 07:34:35,2,2019-11-01,2019-11-02,320100.00,1
12,2019-10-28 15:44:01,2,2019-11-01,2019-11-06,4897530.00,1
41,2019-10-16 07:41:01,2,2019-11-01,2019-11-10,911800.00,1
72,2019-05-06 23:13:56,2,2019-11-01,2019-11-02,435268.00,1
110,2019-10-16 21:24:32,2,2019-11-02,2019-11-07,970000.00,1
...,...,...,...,...,...,...
10649,2020-03-21 04:05:38,2,2020-03-21,2020-03-23,2313450.00,1
10658,2020-03-14 09:14:38,2,2020-03-21,2020-03-24,48.50,2
10668,2020-03-06 06:40:05,2,2020-03-21,2020-03-23,938475.00,1
10707,2020-03-21 07:40:15,2,2020-03-22,2020-03-27,3104000.00,1


# Submission

In [22]:
revenue_per_day.to_csv('revenue_per_day.csv', index=False)